<a href="https://colab.research.google.com/github/alvivar/llama2-googlecolab/blob/main/Llama2_from_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From Huggingface

In [1]:
# Classic transformers
!pip install transformers

# Optimizers
!pip install accelerate
!pip install xformers

# Fine tunning
!pip install trl
!pip install peft
!pip install bitsandbytes
!git clone https://github.com/lvwerra/trl

!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.0 MB/s eta 0:00:00
fatal: destination path 'trl' already exists and is not an empty directory.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from

# Downloads if needed

In [2]:
import requests
from tqdm import tqdm

def download_file_with_progress(url, destination_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(destination_path, 'wb') as file, tqdm(
        desc=destination_path,
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as progress_bar:
        for data in response.iter_content(chunk_size=1024):
            file.write(data)
            progress_bar.update(len(data))

url = "https://www.dropbox.com/s/dkj1dyc2ro7053h/pico8.psd?dl=1"
destination_path = "pico8.psd"

download_file_with_progress(url, destination_path)

pico8.psd: 100%|██████████| 55.9k/55.9k [00:00<00:00, 20.2MB/s]


# Prepare the pipeline from the pretrained model

In [3]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# The prompt

In [ ]:
sequences = pipeline(
    'Enumerate countries ranked by diversity.\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=256,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

# Finetunning with SFTTrainer from trl

In [ ]:
# timdettmers/openassistant-guanaco
# roneneldan/TinyStories

!python trl/examples/scripts/sft_trainer.py \
    --model_name meta-llama/Llama-2-7b-hf \
    --dataset_name timdettmers/openassistant-guanaco \
    --load_in_4bit \
    --use_peft \
    --batch_size 4 \
    --gradient_accumulation_steps 2
